# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [1]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Set headers to mimic a real browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}



In [4]:
def convert_duration_to_minutes(duration_text):

    # if the duration_text doesn't contain hours or minutes, those values will default to 0.
    hours = 0
    minutes = 0

    # extract the hours and minutes from duration_text
    hours_match = re.search(r'(\d+)h', duration_text)
    minutes_match = re.search(r'(\d+)m', duration_text)

    if hours_match:
        hours = int(hours_match.group(1)) # the group method is to retrieve the matched digit. Output: 1
    if minutes_match:
        minutes = int(minutes_match.group(1)) 

    duration = hours * 60 + minutes

    return duration

In [7]:
def scrape_imdb(title_type, user_rating, start_date, end_date):

    url = f'https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating},10'
    #print(f"URL: {url}")  # Print URL for debugging
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract movie data
    movies = soup.find_all('li', class_="ipc-metadata-list-summary-item")
    #print(f"Number of movies found: {len(movies)}")
    
    if not movies:
        print("No movies found. Please check the URL or selectors.")
        return pd.DataFrame(columns=["Movie Nr", "Title", "Year", "Rating", "Runtime (min)", "Genre", "Description", "Director", "Stars", "Votes"])

    # Prepare lists to store the data
    movie_data = {
        "Movie Nr": [],
        "Title": [],
        "Year": [],
        "Rating": [],
        "Duration (min)": [],
        # "Genre": [],
        "Description": [],
        # "Director": [],
        # "Stars": [],
        "Votes": []
    }
    
    for idx, movie in enumerate(movies):
              # Movie Number
        movie_data["Movie Nr"].append(idx + 1)
        
        # Title
        title = movie.h3.text 
        movie_data["Title"].append(title)
        
        # Year
        year = movie.find('span', class_="sc-b189961a-8 kLaxqf dli-title-metadata-item").text
        year = int(re.search(r'\d{4}', year).group())
        movie_data["Year"].append(year)
        
        # Rating
        rating = movie.find('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating').text.split('\xa0')[0]
        movie_data["Rating"].append(float(rating))
        
        # Duration
        runtime = movie.find_all('span', class_="sc-b189961a-8 kLaxqf dli-title-metadata-item")[1].text
        duration = convert_duration_to_minutes(runtime)
        movie_data["Duration (min)"].append(duration)
        
        # Genre
        # genre = movie.find('span', class_='genre').text.strip()
        # movie_data["Genre"].append(genre)
        
        # Description
        description = movie.find('div', class_="ipc-html-content-inner-div").text.strip()
        movie_data["Description"].append(description)

        # # Director and Stars
        # people = movie.find('p', class_='').text.strip().split('|')
        # director = people[0].replace('Director:', '').replace('Directors:', '').strip()
        # stars = people[1].replace('Stars:', '').strip() if len(people) > 1 else ''
        # movie_data["Director"].append(director)
        # movie_data["Stars"].append(stars)
        
        # Votes
        votes = movie.find('span', class_="sc-b0901df4-0 bcQdDJ metacritic-score-box").text
        movie_data["Votes"].append(int(votes))
    
    # Create a DataFrame from the movie data
    df = pd.DataFrame(movie_data)
    
    return df


In [8]:
title_type = 'feature'
start_date = '1990-01-01'
end_date = '1992-12-31'
user_rating = '7.5'

df = scrape_imdb(title_type, user_rating, start_date, end_date)
print(df)

    Movie Nr                          Title  Year  Rating  Duration (min)  \
0          1    1. The Silence of the Lambs  1991     8.6             118   
1          2                  2. Goodfellas  1990     8.7             145   
2          3          3. Dances with Wolves  1990     8.0             181   
3          4  4. Terminator 2: Judgment Day  1991     8.6             137   
4          5              5. A Few Good Men  1992     7.7             138   
5          6            6. Scent of a Woman  1992     8.0             156   
6          7                7. Total Recall  1990     7.5             113   
7          8              8. Reservoir Dogs  1992     8.3              99   
8          9                  9. Home Alone  1990     7.7             103   
9         10            10. My Cousin Vinny  1992     7.6             120   
10        11                     11. Misery  1990     7.8             107   
11        12                 12. Unforgiven  1992     8.2             130   

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here